In [77]:
import os
import sys
import tensorflow as tf

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from ml.models.three_d import c3d

In [39]:
import os
import csv
import numpy as np
import random
from keras.preprocessing.image import ImageDataGenerator
from google.cloud import storage
from etl.lib import cloud_management

In [40]:
BLACKLIST = []
        
# Delete all content in tmp/npy/
filelist = [f for f in os.listdir('tmp/npy')]
for f in filelist:
    os.remove(os.path.join('tmp/npy', f))

# Access Google Cloud Storage
gcs_client = storage.Client.from_service_account_json(
    '/home/harold_triedman/elvo-analysis/credentials/client_secret.json'
)
bucket = gcs_client.get_bucket('elvos')

# Get label data from Google Cloud Storage
blob = storage.Blob('augmented_annotated_labels.csv', bucket)
blob.download_to_filename('tmp/augmented_annotated_labels.csv')
prelim_label_data = {}
with open('tmp/augmented_annotated_labels.csv', 'r') as pos_file:
    reader = csv.reader(pos_file, delimiter=',')
    for row in reader:
        if row[1] != 'Unnamed: 0':
            prelim_label_data[row[1]] = int(row[2])


In [41]:
# Get all of the positives from the label data
positive_label_data = {}
print('getting positive labels')
for id_, label in list(prelim_label_data.items()):
    if label == 1 and '_' in id_:
        positive_label_data[id_] = label

getting positive labels


In [42]:
# Get 12096 random negatives from the label data to feed into our generator
negative_counter = 0
negative_label_data = {}
print(f"getting 14500 random negative labels")
while negative_counter < 14500:
    id_, label = random.choice(list(prelim_label_data.items()))
    if label == 0:
        negative_label_data[id_] = label
        del prelim_label_data[id_]
        negative_counter += 1

getting 12168 random negative labels


In [43]:
chunks = []
labels = []

In [44]:
i = 1
for id_, label in list(negative_label_data.items()):
    print(i, 'chunk_data/normal/negative/' + id_ + '.npy')
    blob = bucket.get_blob('chunk_data/normal/negative/' + id_ + '.npy')
    arr = cloud_management.download_array(blob)
    if arr.shape == (32, 32, 32):
        arr = np.expand_dims(arr, axis=-1)
        chunks.append(arr)
        labels.append(label)
        i += 1

1 chunk_data/normal/negative/14K72RNNXGQ8ESXY469.npy
2 chunk_data/normal/negative/8GW4B6QKOW3ALQCA777.npy
3 chunk_data/normal/negative/ALOUY4SF3BQKXQCZ927.npy
3 chunk_data/normal/negative/THOT71FJ6S8OS792571.npy
4 chunk_data/normal/negative/6XB4FHWAXDVQ79NK776.npy
5 chunk_data/normal/negative/1SDZGHSOKOCXJ82G266.npy
6 chunk_data/normal/negative/DL1FQ4DQMZXMCR9Y777.npy
7 chunk_data/normal/negative/BCNW7CON2NV3XNKA559.npy
7 chunk_data/normal/negative/LVOHLK3FOHWE66BP435.npy
8 chunk_data/normal/negative/OAEVXX4IFAX5IYOR207.npy
9 chunk_data/normal/negative/HFESZX9P9R6TLSSW808.npy
10 chunk_data/normal/negative/0QPSB9IO98216B98765.npy
11 chunk_data/normal/negative/ROMHUDLTCOEXUQCA3.npy
12 chunk_data/normal/negative/CJ8TW38Y0NL2YDAR41.npy
12 chunk_data/normal/negative/VVVMGD8KR33MD3SA626.npy
13 chunk_data/normal/negative/GBXENHYLRVZ5DTHI117.npy
14 chunk_data/normal/negative/OAEVXX4IFAX5IYOR149.npy
15 chunk_data/normal/negative/NWO33W453F6ZJ4BU164.npy
16 chunk_data/normal/negative/OQWBV4PPVHOO

134 chunk_data/normal/negative/QIA74LHB1XWMP523346.npy
135 chunk_data/normal/negative/POSX6PS6LR34HENB179.npy
136 chunk_data/normal/negative/0MTDDGCF20DKOR59284.npy
137 chunk_data/normal/negative/0MTDDGCF20DKOR59356.npy
138 chunk_data/normal/negative/JSV6UEFKJAVM6OGO105.npy
138 chunk_data/normal/negative/2JMHB4LET9UMEJKR499.npy
139 chunk_data/normal/negative/8XJTSY1KAY150OIS465.npy
140 chunk_data/normal/negative/8MVXENAM66B4AWIF136.npy
141 chunk_data/normal/negative/XDNEWPTKFLCWWUSV285.npy
142 chunk_data/normal/negative/R79UN6BZQXMD6MXR237.npy
143 chunk_data/normal/negative/AGTDPQEU54ODFAUD876.npy
144 chunk_data/normal/negative/C5A4R35NQOHJOCDZ28.npy
145 chunk_data/normal/negative/PQ918CDGQIFXXXN1524.npy
146 chunk_data/normal/negative/JJEB6MFMSB6UAKHU464.npy
147 chunk_data/normal/negative/4GN9DK6CIQ0E7HI239.npy
147 chunk_data/normal/negative/3AHS1SYCSNOBIDU5338.npy
148 chunk_data/normal/negative/ADKFJYDHLE7M3XE684.npy
149 chunk_data/normal/negative/MSZB67BPZ32ILNLK56.npy
149 chunk_data

263 chunk_data/normal/negative/LK60A6W87BBO0GSB46.npy
264 chunk_data/normal/negative/NELY4JKZT4KXHG1T360.npy
265 chunk_data/normal/negative/GJ35BUS1EQT27H5F348.npy
266 chunk_data/normal/negative/IXRSXXZI0S6L0EJI454.npy
267 chunk_data/normal/negative/BSXCZXMR5T9ANRE107.npy
268 chunk_data/normal/negative/IJBSBL7IRLNRVXUC235.npy
269 chunk_data/normal/negative/AEPHF1MPR9U7LJN535.npy
270 chunk_data/normal/negative/MJFHYCVEV2GTJD3N222.npy
271 chunk_data/normal/negative/WAY57YXODQSQEIDD459.npy
271 chunk_data/normal/negative/RYNCGNVRZH2LB7BN18.npy
272 chunk_data/normal/negative/I8BO60JRA49C2P5560.npy
273 chunk_data/normal/negative/BWDRAF5ZOQPFOOL742.npy
274 chunk_data/normal/negative/XWC4D3BST0D6WMOT46.npy
275 chunk_data/normal/negative/LCS6AB1M7X4ZP854174.npy
275 chunk_data/normal/negative/FTAKNRFGOGVS632K416.npy
276 chunk_data/normal/negative/0VU79X3RK3L2GY8G635.npy
276 chunk_data/normal/negative/QGEQ4IFSRIA3RBVB301.npy
277 chunk_data/normal/negative/MYB72GJF0W90NZWH155.npy
278 chunk_data/no

394 chunk_data/normal/negative/JT7TUT2NRR0DPZZF81.npy
395 chunk_data/normal/negative/KS626OB4OI9MWE8F452.npy
396 chunk_data/normal/negative/OLYVZJAPCZJQWGIE651.npy
397 chunk_data/normal/negative/Q8LO6RYLTPQQKJO8323.npy
398 chunk_data/normal/negative/HL1S6O6WJW7IHXIY165.npy
399 chunk_data/normal/negative/21A1B91HWFSGQOAU484.npy
400 chunk_data/normal/negative/LIJBIVVBI6TJCT6Z597.npy
401 chunk_data/normal/negative/4UXRRH5MVZVQ1HQW596.npy
402 chunk_data/normal/negative/TFIG39JA77W6USD3191.npy
402 chunk_data/normal/negative/9G0Y9ORHYQF87HGL756.npy
403 chunk_data/normal/negative/EXOIBZU8JEIWNCHS120.npy
404 chunk_data/normal/negative/UKOHM6O57N37U2E2148.npy
405 chunk_data/normal/negative/DZOMCPYXT1TY592W111.npy
405 chunk_data/normal/negative/YGJSUAFCVJ52LIRD125.npy
405 chunk_data/normal/negative/R51O91ZJGN3T0FGI193.npy
405 chunk_data/normal/negative/XHF16O9O7LFK1JNI357.npy
406 chunk_data/normal/negative/8ZK7DFZXQM8IK5BL70.npy
407 chunk_data/normal/negative/STCSWQHX4UN23CDK333.npy
408 chunk_da

533 chunk_data/normal/negative/YRPLT2NPTAFJ25RR330.npy
534 chunk_data/normal/negative/MJFHYCVEV2GTJD3N47.npy
534 chunk_data/normal/negative/TFLSOP3FLUBQDLL2243.npy
535 chunk_data/normal/negative/YSA7ZF8QWN9M5TK634.npy
536 chunk_data/normal/negative/Y4LKD502EZ8ICYLI347.npy
537 chunk_data/normal/negative/64V9V1DQO6O3JJ8F58.npy
538 chunk_data/normal/negative/FLSUE7ZIXI6HSJWN481.npy
538 chunk_data/normal/negative/7E6EK7VVSNAESVYS875.npy
538 chunk_data/normal/negative/NHCZX3GOEMPDDHW376.npy
538 chunk_data/normal/negative/GFRJ7SPGIEJVXVRS1364.npy
539 chunk_data/normal/negative/ZOROHVT5JZVIBEEX75.npy
539 chunk_data/normal/negative/SAQ39AN6XOILQY1D532.npy
540 chunk_data/normal/negative/G3VX8IZCRNCJRP46247.npy
540 chunk_data/normal/negative/0R1R98REO7SARJ06963.npy
541 chunk_data/normal/negative/SZXTZ3LQ5MBFDFXR404.npy
542 chunk_data/normal/negative/XZUUUC426GUXSCGJ226.npy
543 chunk_data/normal/negative/DM6WEZI7LKF1NXUA20.npy
544 chunk_data/normal/negative/WI171X7KEOFWUIUO500.npy
545 chunk_data/

...

10964 chunk_data/normal/negative/P0WH1L0BA1YZXOGW2217.npy
10965 chunk_data/normal/negative/L66E2921S3O1MURX267.npy
10966 chunk_data/normal/negative/8TLM2DUBYEE2GDH0846.npy
10966 chunk_data/normal/negative/NRD1J2CBBXXGPNFL266.npy
10967 chunk_data/normal/negative/76KZMJS23TOIPULM387.npy
10968 chunk_data/normal/negative/COESQX3BUKOI1H5B119.npy
10969 chunk_data/normal/negative/BIASJO43N3A7Z63W20.npy
10970 chunk_data/normal/negative/JCWGEJIJO2LDRY2U624.npy
10971 chunk_data/normal/negative/R51O91ZJGN3T0FGI22.npy
10972 chunk_data/normal/negative/POYYSD3RSJFBKSLQ29.npy
10973 chunk_data/normal/negative/GTSAGN2MRE5D0AZI452.npy
10974 chunk_data/normal/negative/IYDXJTFVWJEX36DO475.npy
10975 chunk_data/normal/negative/3LS918JYONAZ7EUN290.npy
10976 chunk_data/normal/negative/I2XXOTQBYCGCYHXS664.npy
10977 chunk_data/normal/negative/ZQYME7XWFWQXXOXA435.npy
10978 chunk_data/normal/negative/ASI5JI7DJLB3RKFS342.npy
10979 chunk_data/normal/negative/1BBPJ3U25RYGSRSS858.npy
10980 chunk_data/normal/negative/

11095 chunk_data/normal/negative/RRZGAIOEIS6WPXDT833.npy
11096 chunk_data/normal/negative/UN6XC6HYXO0RDKU6197.npy
11096 chunk_data/normal/negative/ASI5JI7DJLB3RKFS97.npy
11097 chunk_data/normal/negative/ZOROHVT5JZVIBEEX276.npy
11098 chunk_data/normal/negative/2JMHB4LET9UMEJKR190.npy
11098 chunk_data/normal/negative/GCSZPU5LPJII8FGU205.npy
11099 chunk_data/normal/negative/C67K82FMKN435ZNB266.npy
11100 chunk_data/normal/negative/DXOUKNW7AXCZFUIL870.npy
11100 chunk_data/normal/negative/IDUHFQWVS4WKXPC5233.npy
11101 chunk_data/normal/negative/GFRJ7SPGIEJVXVRS2714.npy
11102 chunk_data/normal/negative/ZOROHVT5JZVIBEEX209.npy
11103 chunk_data/normal/negative/FK3914JSRYGJ80AO676.npy
11103 chunk_data/normal/negative/SUENUHMZ5JN7IWJE1375.npy
11104 chunk_data/normal/negative/DRS5DZGN5PVUZX4Y285.npy
11105 chunk_data/normal/negative/COESQX3BUKOI1H5B368.npy
11106 chunk_data/normal/negative/65HBYCJJL7IS4B4E267.npy
11107 chunk_data/normal/negative/BGK2KBB8KXRWI65O1791.npy
11107 chunk_data/normal/negat

11218 chunk_data/normal/negative/EURBSIKPYU66GBRW91.npy
11219 chunk_data/normal/negative/NJPOTXDCGONCOU7C400.npy
11220 chunk_data/normal/negative/AQCA8X3KQ99EFUQK453.npy
11221 chunk_data/normal/negative/IETROE62UNT5QVWS240.npy
11222 chunk_data/normal/negative/AOY7EXEF3H62XKG4454.npy
11223 chunk_data/normal/negative/U5ZUK9GYCAM4FYK4877.npy
11224 chunk_data/normal/negative/QJFMNVKKVZXKBROO110.npy
11225 chunk_data/normal/negative/VFHSASVT4EXZWAOS670.npy
11226 chunk_data/normal/negative/IETROE62UNT5QVWS340.npy
11227 chunk_data/normal/negative/ZCTE57VMPCGWDG75199.npy
11228 chunk_data/normal/negative/PCNMFAZL5VWWK7RP189.npy
11228 chunk_data/normal/negative/8SREEDHBQF4ZGA02680.npy
11228 chunk_data/normal/negative/WWEFFBIMLZ3KLQVZ549.npy
11229 chunk_data/normal/negative/4WGQSTEUD1IWW2LT524.npy
11230 chunk_data/normal/negative/IWU76ETUE9UIZN8O79.npy
11231 chunk_data/normal/negative/A096PH8X9XCTJECG140.npy
11231 chunk_data/normal/negative/G37FBCOEGUZFY138556.npy
11232 chunk_data/normal/negative/

11339 chunk_data/normal/negative/JYI7KFRTX8J9V5XI674.npy
11339 chunk_data/normal/negative/X18DVJEHEBZK43JN618.npy
11340 chunk_data/normal/negative/GFRJ7SPGIEJVXVRS771.npy
11341 chunk_data/normal/negative/4FZ9SMCGR26NN28M825.npy
11342 chunk_data/normal/negative/7DFPGGKYXKBYDGNT484.npy
11343 chunk_data/normal/negative/LOGIP43ZSLSUA5SZ120.npy
11344 chunk_data/normal/negative/KYSXQX4HKTWAYLNB347.npy
11345 chunk_data/normal/negative/JLHG7VESTU3GH2KX510.npy
11345 chunk_data/normal/negative/NJZN2TNXUUFRZ3GZ526.npy
11346 chunk_data/normal/negative/BVYSR54MKT5SPMSU37.npy
11347 chunk_data/normal/negative/8XJTSY1KAY150OIS275.npy
11348 chunk_data/normal/negative/UQXCE0KMLR3C4058661.npy
11349 chunk_data/normal/negative/OXRK4I6CTXCENAZV477.npy
11350 chunk_data/normal/negative/UI0XEE9RCAEFSYP2481.npy
11351 chunk_data/normal/negative/POYYSD3RSJFBKSLQ161.npy
11352 chunk_data/normal/negative/9AF84HK0K5CG471Y922.npy
11353 chunk_data/normal/negative/KK0ZHCZR9V87OESW14.npy
11354 chunk_data/normal/negative/

11466 chunk_data/normal/negative/JUOV5YWN6IS7OXIJ162.npy
11467 chunk_data/normal/negative/M66OIOML76PWORZP83.npy
11468 chunk_data/normal/negative/TITRFCZEPFJKM3NY55.npy
11469 chunk_data/normal/negative/3XVH784Y29GVDBHV85.npy
11470 chunk_data/normal/negative/SE3O4E4F4Z70C75B458.npy
11471 chunk_data/normal/negative/PKSQ49QA03C0GTJ8249.npy
11472 chunk_data/normal/negative/G4792R3E9O3KQJ5M421.npy
11473 chunk_data/normal/negative/FUVL4PLSUHRCEAHC618.npy
11474 chunk_data/normal/negative/VJNCTEEYR7U4HIQU474.npy
11475 chunk_data/normal/negative/PAHYRPTLLQOU2FTB626.npy
11476 chunk_data/normal/negative/OAEVXX4IFAX5IYOR127.npy
11477 chunk_data/normal/negative/H8L1IQDRQK22ZWJ3218.npy
11478 chunk_data/normal/negative/SAQ39AN6XOILQY1D680.npy
11479 chunk_data/normal/negative/G06KIRHKXWJZV4JL192.npy
11479 chunk_data/normal/negative/BGK2KBB8KXRWI65O600.npy
11480 chunk_data/normal/negative/KD1MCH6J7162T5QA248.npy
11481 chunk_data/normal/negative/ZOROHVT5JZVIBEEX145.npy
11482 chunk_data/normal/negative/T

In [45]:
i = 1
for id_, label in list(positive_label_data.items()):
    print(i, 'chunk_data/normal/positive/' + id_  + '.npy')
    blob = bucket.get_blob('chunk_data/normal/positive/' + id_ + '.npy')
    arr = cloud_management.download_array(blob)
    if arr.shape == (32, 32, 32):
        arr = np.expand_dims(arr, axis=-1)
        chunks.append(arr)
        labels.append(label)
        i += 1

1 chunk_data/normal/positive/04IOS24JP70LHBGB508_1.npy
2 chunk_data/normal/positive/04IOS24JP70LHBGB508_2.npy
3 chunk_data/normal/positive/04IOS24JP70LHBGB508_3.npy
4 chunk_data/normal/positive/04IOS24JP70LHBGB508_4.npy
5 chunk_data/normal/positive/04IOS24JP70LHBGB508_5.npy
6 chunk_data/normal/positive/04IOS24JP70LHBGB508_6.npy
7 chunk_data/normal/positive/04IOS24JP70LHBGB508_7.npy
8 chunk_data/normal/positive/04IOS24JP70LHBGB508_8.npy
9 chunk_data/normal/positive/04IOS24JP70LHBGB508_9.npy
10 chunk_data/normal/positive/04IOS24JP70LHBGB508_10.npy
11 chunk_data/normal/positive/04IOS24JP70LHBGB508_11.npy
12 chunk_data/normal/positive/04IOS24JP70LHBGB508_12.npy
13 chunk_data/normal/positive/04IOS24JP70LHBGB508_13.npy
14 chunk_data/normal/positive/04IOS24JP70LHBGB508_14.npy
15 chunk_data/normal/positive/04IOS24JP70LHBGB508_15.npy
16 chunk_data/normal/positive/04IOS24JP70LHBGB508_16.npy
17 chunk_data/normal/positive/04IOS24JP70LHBGB508_17.npy
18 chunk_data/normal/positive/04IOS24JP70LHBGB508

146 chunk_data/normal/positive/0MG5EK1O64CPS887338_2.npy
147 chunk_data/normal/positive/0MG5EK1O64CPS887338_3.npy
148 chunk_data/normal/positive/0MG5EK1O64CPS887338_4.npy
149 chunk_data/normal/positive/0MG5EK1O64CPS887338_5.npy
150 chunk_data/normal/positive/0MG5EK1O64CPS887338_6.npy
151 chunk_data/normal/positive/0MG5EK1O64CPS887338_7.npy
152 chunk_data/normal/positive/0MG5EK1O64CPS887338_8.npy
153 chunk_data/normal/positive/0MG5EK1O64CPS887338_9.npy
154 chunk_data/normal/positive/0MG5EK1O64CPS887338_10.npy
155 chunk_data/normal/positive/0MG5EK1O64CPS887338_11.npy
156 chunk_data/normal/positive/0MG5EK1O64CPS887338_12.npy
157 chunk_data/normal/positive/0MG5EK1O64CPS887338_13.npy
158 chunk_data/normal/positive/0MG5EK1O64CPS887338_14.npy
159 chunk_data/normal/positive/0MG5EK1O64CPS887338_15.npy
160 chunk_data/normal/positive/0MG5EK1O64CPS887338_16.npy
161 chunk_data/normal/positive/0MG5EK1O64CPS887338_17.npy
162 chunk_data/normal/positive/0MG5EK1O64CPS887338_18.npy
163 chunk_data/normal/

291 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_3.npy
292 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_4.npy
293 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_5.npy
294 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_6.npy
295 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_7.npy
296 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_8.npy
297 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_9.npy
298 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_10.npy
299 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_11.npy
300 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_12.npy
301 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_13.npy
302 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_14.npy
303 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_15.npy
304 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_16.npy
305 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_17.npy
306 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_18.npy
307 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_19.npy
308 chunk_data/normal

435 chunk_data/normal/positive/2A617MJ3LGPCXRTK598_3.npy
436 chunk_data/normal/positive/2A617MJ3LGPCXRTK598_4.npy
437 chunk_data/normal/positive/2A617MJ3LGPCXRTK598_5.npy
438 chunk_data/normal/positive/2A617MJ3LGPCXRTK598_6.npy
439 chunk_data/normal/positive/2A617MJ3LGPCXRTK598_7.npy
440 chunk_data/normal/positive/2A617MJ3LGPCXRTK598_8.npy
441 chunk_data/normal/positive/2A617MJ3LGPCXRTK598_9.npy
442 chunk_data/normal/positive/2A617MJ3LGPCXRTK598_10.npy
443 chunk_data/normal/positive/2A617MJ3LGPCXRTK598_11.npy
444 chunk_data/normal/positive/2A617MJ3LGPCXRTK598_12.npy
445 chunk_data/normal/positive/2A617MJ3LGPCXRTK598_13.npy
446 chunk_data/normal/positive/2A617MJ3LGPCXRTK598_14.npy
447 chunk_data/normal/positive/2A617MJ3LGPCXRTK598_15.npy
448 chunk_data/normal/positive/2A617MJ3LGPCXRTK598_16.npy
449 chunk_data/normal/positive/2A617MJ3LGPCXRTK598_17.npy
450 chunk_data/normal/positive/2A617MJ3LGPCXRTK598_18.npy
451 chunk_data/normal/positive/2A617MJ3LGPCXRTK598_19.npy
452 chunk_data/normal

580 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_4.npy
581 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_5.npy
582 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_6.npy
583 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_7.npy
584 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_8.npy
585 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_9.npy
586 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_10.npy
587 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_11.npy
588 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_12.npy
589 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_13.npy
590 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_14.npy
591 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_15.npy
592 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_16.npy
593 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_17.npy
594 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_18.npy
595 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_19.npy
596 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_20.npy
597 chunk_data/norma

...

11558 chunk_data/normal/positive/YNLMIJ5S24AYYZXN312_14.npy
11559 chunk_data/normal/positive/YNLMIJ5S24AYYZXN312_15.npy
11560 chunk_data/normal/positive/YNLMIJ5S24AYYZXN312_16.npy
11561 chunk_data/normal/positive/YNLMIJ5S24AYYZXN312_17.npy
11562 chunk_data/normal/positive/YNLMIJ5S24AYYZXN312_18.npy
11563 chunk_data/normal/positive/YNLMIJ5S24AYYZXN312_19.npy
11564 chunk_data/normal/positive/YNLMIJ5S24AYYZXN312_20.npy
11565 chunk_data/normal/positive/YNLMIJ5S24AYYZXN312_21.npy
11566 chunk_data/normal/positive/YNLMIJ5S24AYYZXN312_22.npy
11567 chunk_data/normal/positive/YNLMIJ5S24AYYZXN312_23.npy
11568 chunk_data/normal/positive/YNLMIJ5S24AYYZXN312_24.npy
11569 chunk_data/normal/positive/YP3O7PTO0AG3Q9LN311_1.npy
11570 chunk_data/normal/positive/YP3O7PTO0AG3Q9LN311_2.npy
11571 chunk_data/normal/positive/YP3O7PTO0AG3Q9LN311_3.npy
11572 chunk_data/normal/positive/YP3O7PTO0AG3Q9LN311_4.npy
11573 chunk_data/normal/positive/YP3O7PTO0AG3Q9LN311_5.npy
11574 chunk_data/normal/positive/YP3O7PTO0AG3

11696 chunk_data/normal/positive/YYNFU45K2WIMXWE4404_8.npy
11697 chunk_data/normal/positive/YYNFU45K2WIMXWE4404_9.npy
11698 chunk_data/normal/positive/YYNFU45K2WIMXWE4404_10.npy
11699 chunk_data/normal/positive/YYNFU45K2WIMXWE4404_11.npy
11700 chunk_data/normal/positive/YYNFU45K2WIMXWE4404_12.npy
11701 chunk_data/normal/positive/YYNFU45K2WIMXWE4404_13.npy
11702 chunk_data/normal/positive/YYNFU45K2WIMXWE4404_14.npy
11703 chunk_data/normal/positive/YYNFU45K2WIMXWE4404_15.npy
11704 chunk_data/normal/positive/YYNFU45K2WIMXWE4404_16.npy
11705 chunk_data/normal/positive/YYNFU45K2WIMXWE4404_17.npy
11706 chunk_data/normal/positive/YYNFU45K2WIMXWE4404_18.npy
11707 chunk_data/normal/positive/YYNFU45K2WIMXWE4404_19.npy
11708 chunk_data/normal/positive/YYNFU45K2WIMXWE4404_20.npy
11709 chunk_data/normal/positive/YYNFU45K2WIMXWE4404_21.npy
11710 chunk_data/normal/positive/YYNFU45K2WIMXWE4404_22.npy
11711 chunk_data/normal/positive/YYNFU45K2WIMXWE4404_23.npy
11712 chunk_data/normal/positive/YYNFU45K2

11835 chunk_data/normal/positive/ZC9H37RWIQ90483S304_3.npy
11836 chunk_data/normal/positive/ZC9H37RWIQ90483S304_4.npy
11837 chunk_data/normal/positive/ZC9H37RWIQ90483S304_5.npy
11838 chunk_data/normal/positive/ZC9H37RWIQ90483S304_6.npy
11839 chunk_data/normal/positive/ZC9H37RWIQ90483S304_7.npy
11840 chunk_data/normal/positive/ZC9H37RWIQ90483S304_8.npy
11841 chunk_data/normal/positive/ZC9H37RWIQ90483S304_9.npy
11842 chunk_data/normal/positive/ZC9H37RWIQ90483S304_10.npy
11843 chunk_data/normal/positive/ZC9H37RWIQ90483S304_11.npy
11844 chunk_data/normal/positive/ZC9H37RWIQ90483S304_12.npy
11845 chunk_data/normal/positive/ZC9H37RWIQ90483S304_13.npy
11846 chunk_data/normal/positive/ZC9H37RWIQ90483S304_14.npy
11847 chunk_data/normal/positive/ZC9H37RWIQ90483S304_15.npy
11848 chunk_data/normal/positive/ZC9H37RWIQ90483S304_16.npy
11849 chunk_data/normal/positive/ZC9H37RWIQ90483S304_17.npy
11850 chunk_data/normal/positive/ZC9H37RWIQ90483S304_18.npy
11851 chunk_data/normal/positive/ZC9H37RWIQ9048

11974 chunk_data/normal/positive/ZSXFDYQ4ST4Z6SRF262_22.npy
11975 chunk_data/normal/positive/ZSXFDYQ4ST4Z6SRF262_23.npy
11976 chunk_data/normal/positive/ZSXFDYQ4ST4Z6SRF262_24.npy
11977 chunk_data/normal/positive/ZUEK5YSS7CITVWIP468_1.npy
11978 chunk_data/normal/positive/ZUEK5YSS7CITVWIP468_2.npy
11979 chunk_data/normal/positive/ZUEK5YSS7CITVWIP468_3.npy
11980 chunk_data/normal/positive/ZUEK5YSS7CITVWIP468_4.npy
11981 chunk_data/normal/positive/ZUEK5YSS7CITVWIP468_5.npy
11982 chunk_data/normal/positive/ZUEK5YSS7CITVWIP468_6.npy
11983 chunk_data/normal/positive/ZUEK5YSS7CITVWIP468_7.npy
11984 chunk_data/normal/positive/ZUEK5YSS7CITVWIP468_8.npy
11985 chunk_data/normal/positive/ZUEK5YSS7CITVWIP468_9.npy
11986 chunk_data/normal/positive/ZUEK5YSS7CITVWIP468_10.npy
11987 chunk_data/normal/positive/ZUEK5YSS7CITVWIP468_11.npy
11988 chunk_data/normal/positive/ZUEK5YSS7CITVWIP468_12.npy
11989 chunk_data/normal/positive/ZUEK5YSS7CITVWIP468_13.npy
11990 chunk_data/normal/positive/ZUEK5YSS7CITVWIP

12113 chunk_data/normal/positive/ZXXMMCGK6ANRKLFD372_17.npy
12114 chunk_data/normal/positive/ZXXMMCGK6ANRKLFD372_18.npy
12115 chunk_data/normal/positive/ZXXMMCGK6ANRKLFD372_19.npy
12116 chunk_data/normal/positive/ZXXMMCGK6ANRKLFD372_20.npy
12117 chunk_data/normal/positive/ZXXMMCGK6ANRKLFD372_21.npy
12118 chunk_data/normal/positive/ZXXMMCGK6ANRKLFD372_22.npy
12119 chunk_data/normal/positive/ZXXMMCGK6ANRKLFD372_23.npy
12120 chunk_data/normal/positive/ZXXMMCGK6ANRKLFD372_24.npy
12121 chunk_data/normal/positive/ZYKNNBEIY0387GUQ476_1.npy
12122 chunk_data/normal/positive/ZYKNNBEIY0387GUQ476_2.npy
12123 chunk_data/normal/positive/ZYKNNBEIY0387GUQ476_3.npy
12124 chunk_data/normal/positive/ZYKNNBEIY0387GUQ476_4.npy
12125 chunk_data/normal/positive/ZYKNNBEIY0387GUQ476_5.npy
12126 chunk_data/normal/positive/ZYKNNBEIY0387GUQ476_6.npy
12127 chunk_data/normal/positive/ZYKNNBEIY0387GUQ476_7.npy
12128 chunk_data/normal/positive/ZYKNNBEIY0387GUQ476_8.npy
12129 chunk_data/normal/positive/ZYKNNBEIY0387GU

In [52]:
tmp = list(zip(chunks, labels))
random.shuffle(tmp)
chunks, labels = zip(*tmp)

# Split based on validation
print('splitting based on validation split')
X_train = np.asarray(chunks[int(len(chunks) * 0.1):])
y_train = np.asarray(labels[int(len(labels) * 0.1):])

X_val = np.asarray(chunks[:int(len(chunks) * 0.1)])
y_val = np.asarray(labels[:int(len(labels) * 0.1)])

splitting based on validation split


In [53]:
print(len(chunks), len(chunks[0]), len(chunks[0][0]), len(chunks[0][0][0]))

23703 32 32 32


In [54]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [71]:
from keras.optimizers import SGD
from keras import metrics
from keras import backend as K
from keras.callbacks import ModelCheckpoint

LEARN_RATE = 1e-5

In [72]:
def true_positives(y_true, y_pred):
    return K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))


def false_negatives(y_true, y_pred):
    return K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))


def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())


def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [73]:
metrics.true_positives = true_positives
metrics.false_negtives = false_negatives
metrics.sensitivity = sensitivity
metrics.specificity = specificity

In [74]:
model = c3d.C3DBuilder.build()
model.compile(optimizer=SGD(lr=LEARN_RATE, momentum=0.9, nesterov=True),
              loss={"out_class": "binary_crossentropy"},
#               loss='binary_crossentropy'
              metrics={"out_class": [metrics.binary_accuracy,
                                     metrics.binary_crossentropy,
                                     true_positives,
                                     false_negatives,
                                     sensitivity,
                                     specificity]
                      }
             )
#              metrics=['accuracy'])

/home/harold_triedman/elvo-analysis/ml/models/three_d/c3d.py:24: UserWarning: Update your `AveragePooling3D` call to the Keras 2 API: `AveragePooling3D(pool_size=(2, 1, 1), strides=(2, 1, 1), padding="same")`
  strides=(2, 1, 1), border_mode="same")(x)
/home/harold_triedman/elvo-analysis/ml/models/three_d/c3d.py:27: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), activation="relu", name="conv1", strides=(1, 1, 1), padding="same")`
  subsample=(1, 1, 1))(x)
/home/harold_triedman/elvo-analysis/ml/models/three_d/c3d.py:30: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid")`
  name='pool1')(x)
/home/harold_triedman/elvo-analysis/ml/models/three_d/c3d.py:35: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3), activation="relu", name="conv2", strides=(1, 1, 1), padding="same")`
  subsample=(1, 1, 1))(x)
/home/harold_triedman/elvo-

____________________________________________________________________________________________________________________________________________
Layer (type)                                                   Output Shape                                            Param #              
input_1 (InputLayer)                                           (None, 32, 32, 32, 1)                                   0                    
____________________________________________________________________________________________________________________________________________
average_pooling3d_9 (AveragePooling3D)                         (None, 16, 32, 32, 1)                                   0                    
____________________________________________________________________________________________________________________________________________
conv1 (Conv3D)                                                 (None, 16, 32, 32, 64)                                  1792                 
_____________

In [75]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(21333, 32, 32, 32, 1) (21333,)
(2370, 32, 32, 32, 1) (2370,)


In [76]:
mc_callback = ModelCheckpoint(filepath='tmp/c3d_weights.hdf5', verbose=1)
model.fit(
    x=X_train,
    y=y_train,
    epochs=100,
    batch_size=16,
    callbacks=[mc_callback],
    validation_data = (X_val, y_val),
    verbose=1)

Train on 21333 samples, validate on 2370 samples
Epoch 1/100
21333/21333 [==============================] - 90s 4ms/step - loss: 0.3650 - binary_accuracy: 0.8606 - binary_crossentropy: 0.3650 - true_positives: 7.3148 - false_negatives: 0.8820 - sensitivity: 0.8914 - specificity: 0.8269 - val_loss: 0.2695 - val_binary_accuracy: 0.9042 - val_binary_crossentropy: 0.2695 - val_true_positives: 7.4068 - val_false_negatives: 0.9384 - val_sensitivity: 0.8840 - val_specificity: 0.9219

Epoch 00001: saving model to tmp/c3d_weights.hdf5
Epoch 2/100
21333/21333 [==============================] - 88s 4ms/step - loss: 0.1974 - binary_accuracy: 0.9272 - binary_crossentropy: 0.1974 - true_positives: 7.8256 - false_negatives: 0.3713 - sensitivity: 0.9545 - specificity: 0.8990 - val_loss: 0.2015 - val_binary_accuracy: 0.9266 - val_binary_crossentropy: 0.2015 - val_true_positives: 8.1291 - val_false_negatives: 0.2160 - val_sensitivity: 0.9724 - val_specificity: 0.8739

Epoch 00002: saving model to tmp/c3

KeyboardInterrupt: 

In [54]:
import matplotlib.pyplot as plt